# Задание 1

Реализуйте стохастический градиентный спуск для решения линейной регрессии. Исследуйте сходимость с разным размером батча (1 - SGD, 2, $\ldots$, $n - 1$ - Minibatch GD, $n$ - GD из предыдущей работы).

In [2]:
import sys
import os
sys.path.append('..' + '/')

In [3]:
import random
from helper import *

def stochastic_gradient_descent(f, initial_point, learning_rate=0.1, max_epochs=1000, minimum = 0.0, epsilon=1e-5, batch_size=1, apply_min=False):
    """
    Cтохастический градиентный спуск для поиска минимума функции.

    Аргументы:
        f (function): Изначальная функция.
        grad_fn (function): Функция, которая принимает точку и возвращает градиент в этой точке.
        initial_point (list): Начальную точка, с которой начинается поиск.
        learning_rate (float): Скорость обучения или шаг градиентного спуска.
        max_epochs (int): Максимальное количество эпох или итераций для выполнения алгоритма.
        minimum (float): Минимум функции.
        epsilon (float): Малое число, используемое как критерий останова для алгоритма.
        batch_size (int): кол-во координат по которым вычисляется градиент
    Возвращает:
        Кортеж, содержащий найденную минимальную точку, значение функции в этой точке и список всех точек, посещенных во время алгоритма.
    """

    batch_size = min(batch_size, len(initial_point))

    current_point = initial_point.copy() # текущая точка, инициализируется начальной точкой
    current_value = f(current_point) # значение функции в текущей точке
    visited_points = [current_point.copy()] # список посещенных точек, начинается с начальной точки
    for _ in range(max_epochs): # цикл по эпохам
        if abs(current_value - minimum) < epsilon: # если достигнуто достаточно малое значение функции, то останавливаемся
            break
        prev_point = np.copy(current_point) 
        for _ in range(batch_size):
            random_index = random.randint(0, len(current_point)-1) # выбираем случайный индекс измерения
            gradient_random_index = fast_gradient(f, current_point, random_index) # вычисляем градиент в текущей точке в случайном индексе
            current_point[random_index] -= learning_rate * gradient_random_index # обновляем текущую точку

        new_value = f(current_point) # вычисляем значение функции в обновленной точке
        if new_value < current_value: # если значение функции в обновленной точке меньше, чем в предыдущей, то продолжаем движение
            current_value = new_value
        else: # если значение функции больше или не изменилось, то возвращаемся к предыдущей точке
            current_point = prev_point
        visited_points.append(current_point.copy()) # добавляем текущую точку в список посещенных
    return current_point, current_value, visited_points # возвращаем результат работы функции

In [32]:
def linear_regression(X, y, epsilon=1e-5, learning_rate=0.01, max_epochs=1000, apply_min=False):
    # X = file_info.X
    # y = file_info.y
    # a = file_info.real_weight
    # b = file_info.real_bias

    def mse_loss(x):
        weights, bias = x[:-1], x[-1]
        y_pred = np.dot(X, weights) + bias
        mse = np.mean((y.reshape(-1, 1) - y_pred)**2)
        return mse


    # def mse_loss(x):
    #     print(x)

    #     weights, bias = x[:-1], x[-1]
    #     y_pred = np.dot(X, weights) + bias
    #     mse = np.mean((y - y_pred) ** 2)
    #     return mse

    # def mae_loss(x):
    #     weights, bias = x[:-1], x[-1]
    #     y_pred = np.dot(X, weights) + bias
    #     mae = np.mean(np.abs(y - y_pred))
    #     return mae


    f = [mse_loss]
    labels = ['mse_loss']

    # file_info.labels_loss = labels

    weight = 0
    bias = 0

    x0 = np.array([0, 0, 0, 0, 0], dtype=float)

    results = []
    count = []
    loss_history = []
    loss_real = []
    
    for i in range(len(f)):
        point, value, result = stochastic_gradient_descent(f[i], x0, epsilon=epsilon, learning_rate=learning_rate, max_epochs=max_epochs, batch_size=2, apply_min=apply_min)
        results.append(result)
        count.append(len(result))
        loss_history.append([f[i](point) for point in result])
        # loss_real.append(f[i]([a, b]))
    return results, count, loss_history, loss_real, labels


In [39]:
def random_generator(real_weights, real_bias, dots_count=100, variance=1):
    X = np.random.rand(dots_count, len(real_weights))
    y = real_weights * X + real_bias + (np.random.rand(dots_count, len(real_weights)) * 2 * variance - variance)
    return X, y

real_weights = [1, 2, 3, 4]
real_bias = -1
dots_count=50
variance=0.25
X, y = random_generator(real_weights, real_bias, dots_count, variance)

result, count, loss_history, loss_real, labels_loss = linear_regression(X, y, epsilon=0.0085, learning_rate = 0.1, max_epochs=1000, apply_min=True) 

print(result[-1][-1])

def mse_loss(x):
    weights, bias = x[:-1], x[-1]
    y_pred = np.dot(X, weights) + bias
    mse = np.mean((y.reshape(-1, 1) - y_pred)**2)
    return mse

print(mse_loss([1, 2, 3, 4, -1]))
print(mse_loss(result[-1][-1]))

# print_2d_data(f_info, [[[[2, -1]]]], ['Mini-batch'], show_print=True)
# print_loss(loss_history, loss_real, labels_loss)
# print_2d_function(loss_history, labels_loss)

[0.00114844 0.00118456 0.00051563 0.00089304 0.20114024]
17.63332640173388
1.0623710711500731
